In [4]:

!pip install openai
!pip install langchain
!pip install langchain_community
!pip install tiktoken
!pip install faiss-cpu
!pip install langchain-pinecone
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.8 MB/s eta 0:00:00


In [26]:
# Import necessary libraries
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
# Configure the baseline configuration of the OpenAI library for Azure OpenAI Service.
OPENAI_API_KEY = "YOUR KEY HERE"

In [28]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [29]:
llm.invoke("What is EDA?")

'\n\nEDA stands for Exploratory Data Analysis. It is the process of examining and analyzing data in order to discover patterns, relationships, and trends. EDA is an important step in the data analysis process as it allows data scientists to gain a better understanding of the data and its characteristics before performing more complex analyses and modeling. EDA often involves visualizations, statistical summaries, and data cleaning and transformation techniques. It is used in various fields such as data science, machine learning, and business analytics.'

#### using **custom** dataset

In [30]:
pdf_reader = PyPDFLoader("/content/RAGPaper.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, )
chunks = text_splitter.split_documents(documents)

In [31]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Create embeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
db = FAISS.from_documents(documents=chunks, embedding=embeddings)


In [32]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [33]:
chat_history = []
query = """?What is NDA?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




I don't know.


In [34]:
chat_history = []
query = """?What is RAG?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

 RAG stands for Retrieval-Augmented Generation, which is a model that combines retrieval of external knowledge sources with generation to answer questions or complete tasks. It has been shown to outperform previous approaches in various natural language processing tasks and offers benefits such as being more factual and controllable.


## Using the entire PDF as context

If we try to invoke the chain using the PDF as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [2]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("RAGPaper.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research;‡University College London;⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research problems. Pre-\ntrained models with a differentiable access mechanism to explicit 

## Splitting the PDF

Since we can't use the entire PDF as the context for the model, a potential solution is to split the PDF into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question:


Let's start by loading the PDF in memory:

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size. Check [Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/) for more information about different approaches to splitting documents.

For illustration purposes, let's split the PDF into chunks of 100 characters with an overlap of 20 characters and display the first few chunks:

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(pages)[:5]

[Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,'),
 Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,'),
 Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†'),
 Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='†Facebook AI Research;‡University College London;⋆New York University;\nplewis@fb.com\nAbstract'),
 Document(metadata={'source': 'RAGPaper.pdf', 'page': 0}, page_content='Abstract\nLarge pre-trained language models have been shown to store factual knowledge')]

For our specific application, let's use 1000 characters instead:

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

## Finding the relevant chunks

Given a particular question, we need to find the relevant chunks from the transcription to send to the model. Here is where the idea of **embeddings** comes into play.

An embedding is a mathematical representation of the semantic meaning of a word, sentence, or document. It's a projection of a concept in a high-dimensional space. Embeddings have a simple characteristic: The projection of related concepts will be close to each other, while concepts with different meanings will lie far away. You can use the [Cohere's Embed Playground](https://dashboard.cohere.com/playground/embed) to visualize embeddings in two dimensions.

To provide with the most relevant chunks, we can use the embeddings of the question and the chunks of the transcription to compute the similarity between them. We can then select the chunks with the highest similarity to the question and use them as the context for the model:


Let's generate embeddings for an arbitrary query:

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
embedded_query = embeddings.embed_query("What is the full form of RAG?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Embedding length: 1536
[0.005202177057250298, -0.0046028635747289584, 0.017380079817248775, -0.012906137819897572, -0.00036237534568487993, -0.006153412168688117, -0.021533457814425477, -0.010007134821587906, -0.046551294868395245, -0.028530090776087487]


To illustrate how embeddings work, let's first generate the embeddings for two different sentences:

In [15]:
sentence1 = embeddings.embed_query("I love to watch youtube videos")
sentence2 = embeddings.embed_query("What is RAG")
sentence3 = embeddings.embed_query("Who is Roger Federer")

We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to calculate the similarity between the query and each of the sentences:

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]
query_sentence3_similarity = cosine_similarity([embedded_query], [sentence3])[0][0]

print(query_sentence1_similarity)
print(query_sentence2_similarity)
print(query_sentence3_similarity)

0.6816105265861361
0.9277846627962576
0.7198870537004113


## Setting up Pinecone

So far we've used an in-memory vector store. In practice, we need a vector store that can handle large amounts of data and perform similarity searches at scale. For this example, we'll use [Pinecone](https://www.pinecone.io/).

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable `PINECONE_API_KEY`.

Then, we can load the transcription documents into Pinecone:

In [17]:
from pinecone import Pinecone
import os

pc = Pinecone(api_key="3d983071-ee3f-4a33-9793-752b96c8026a")

In [19]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud,region=region)

In [20]:
index_name = 'testindex'

In [21]:
if index_name not in pc.list_indexes().names():
  pc.create_index(index_name, dimension=1536,metric='cosine',spec=spec)

index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Authenticating Pinecone

In [23]:
!pip install pinecone_notebooks

In [24]:
from pinecone_notebooks.colab import Authenticate
Authenticate()

In [35]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(chunks,embeddings, index_name = index_name)

In [36]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x7eb14457d930>)

In [37]:
qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=docsearch.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [38]:
chat_history = []
query = """?What is NDA?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

 I don't know, I don't have enough context to accurately answer this question.



In [39]:
chat_history = []
query = """?What is RAG?"""
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])



RAG stands for Retriever, Reader, and Generator, and it is a model that uses input sequences to retrieve text documents and generate correct answers. It has shown to outperform other models in tasks such as question answering and fact verification, and it offers more control and interpretability due to its grounding in factual knowledge (such as Wikipedia). However, there may be concerns about potential biases in the external knowledge sources used by RAG.
